In [2]:
import taichi as ti
import numpy as np
import math

ti.init(arch=ti.gpu)

screen_res = (800,400)
screen_to_world_ratio = 10.0
boundary = (screen_res[0]/screen_to_world_ratio, screen_res[1]/screen_to_world_ratio)

bg_color = 0x112f41
particle_color = 0x068587
boundary_color = 0xebaca2

num_particles_x = 60
num_particles = num_particles_x * 20
particle_radius = 3.0
particle_radius_in_world = particle_radius / screen_to_world_ratio
board_states = ti.Vector(2, dt=ti.f32)
delta_time = 1.0 / 20.0
epsilon = 1e-5
lambda_epsilon = 100.0
pbf_iter = 5

#PBF params
dim = 2
mass = 1.0
old_positions = ti.Vector(dim, dt=ti.f32)
positions = ti.Vector(dim, dt=ti.f32)
velocities = ti.Vector(dim, dt=ti.f32)
delta_positions = ti.Vector(dim, dt=ti.f32)
rest_density = 1.0
kernel = 1.1
max_num_neighbors = 100
neighbor_particles = ti.var(dt=ti.i32)
num_neighbor_particles = ti.var(dt=ti.i32)
lambdas = ti.var(ti.f32)
delta_q = 0.2 * kernel
k = 0.001

ti.root.dense(ti.i, num_particles).place(old_positions, positions, velocities)
nb_node = ti.root.dense(ti.i, num_particles)
nb_node.place(num_neighbor_particles)
nb_node.dense(ti.j, max_num_neighbors).place(neighbor_particles)
ti.root.dense(ti.i, num_particles).place(lambdas, delta_positions)
ti.root.place(board_states)

@ti.kernel
def mapping(x: ti.template(), y: ti.template()):
    for i in x:
        y[i] = x[i]

@ti.func
def poly6_value(r, h):
    result = 0.0
    if 0 < r and r < h:
        x = (h * h - r * r) / (h * h * h)
        result = x * x * x * 315.0 / 64.0 / np.pi
    return result

@ti.func
def spiky_gradient(r, h):
    r_mod = r.norm()
    result = ti.Vector([0.0,0.0])
    if r_mod > 0 and r_mod < h:
        tmp = (h - r_mod) / h**3
        g_fector = tmp * tmp * (-45.0 / np.pi)
        result = r * g_fector / r_mod
    return result

@ti.func
def confine_position_to_boundary(pos):
    bmin = particle_radius_in_world
    bmax = ti.Vector([board_states[None][0], boundary[1]]) - particle_radius_in_world
    for i in ti.static(range(dim)):
        if pos[i] <= bmin:
            pos[i] = bmin + epsilon * ti.random()
        elif pos[i] >= bmax[i]:
            pos[i] = bmax[i] - epsilon * ti.random()
    return pos

@ti.kernel
def confine_to_boundary():
    for i in positions:
        pos = positions[i]
        positions[i] = confine_position_to_boundary(pos)

@ti.kernel
def apply_gravity():
    for i in positions:
        g = ti.Vector([0.0,-9.8])
        pos = positions[i]
        vel = velocities[i]
        vel += g * delta_time
        pos += vel * delta_time
        positions[i] = confine_position_to_boundary(pos)

@ti.kernel
def find_neighbor_particles():
    for i in positions:
        pos_i = positions[i]
        cnt = 0
        for j in range(num_particles):
            if i != j:
                pos_j = positions[j]
                r = pos_i - pos_j
                r_mod = r.norm()
                if (r_mod > 0 and r_mod < kernel * 1.05) and cnt < num_particles:
                    neighbor_particles[i, cnt] = j
                    cnt += 1
        num_neighbor_particles[i] = cnt

@ti.kernel
def compute_lambdas():
    for i in positions:
        pos_i = positions[i]
        
        grad_i = ti.Vector([0.0,0.0])
        sum_grad_mod = 0.0
        density_constraint = 0.0
        
        for j in range(num_neighbor_particles[i]):
            p_j = neighbor_particles[i, j]
            if p_j >= 0:
                pos_j = positions[p_j]
                r = pos_i - pos_j
                grad_j = spiky_gradient(r, kernel)
                grad_i += grad_j
                sum_grad_mod += grad_j.dot(grad_j)
                r_mod = r.norm()
                density_constraint += poly6_value(r_mod, kernel)
                
        density_constraint = (mass * density_constraint / rest_density) - 1.0
        sum_grad_mod += grad_i.dot(grad_i)
        lambdas[i] = (-density_constraint) / (sum_grad_mod + lambda_epsilon)

@ti.func
def compute_scorr(r):
    tmp = (poly6_value(r.norm(), kernel) / poly6_value(delta_q, kernel))
    tmp = tmp * tmp
    tmp = tmp * tmp
    return (-k) * tmp

@ti.kernel
def compute_delta_positions():
    for i in positions:
        pos_i = positions[i]
        lambda_i = lambdas[i]
        
        delta_pos = ti.Vector([0.0,0.0])
        for j in range(num_neighbor_particles[i]):
            p_j = neighbor_particles[i, j]
            if p_j >= 0:
                pos_j = positions[p_j]
                lambda_j = lambdas[p_j]
                r = pos_i - pos_j
                scorr_value = compute_scorr(r)
                delta_pos += (lambda_i + lambda_j + scorr_value) * spiky_gradient(r, kernel)
                
        delta_positions[i] = delta_pos / rest_density

@ti.kernel
def apply_delta_positions():
    for i in positions:
        positions[i] += delta_positions[i]

@ti.kernel
def update_velocities():
    for i in positions:
        velocities[i] = (positions[i] - old_positions[i]) / delta_time

@ti.kernel
def move_board():
    # probably more accurate to exert force on particles according to hooke's law.
    b = board_states[None]
    b[1] += 1.0
    period = 90
    vel_strength = 8.0
    if b[1] >= 2 * period:
        b[1] = 0
    b[0] += -ti.sin(b[1] * math.pi / period) * vel_strength * delta_time
    board_states[None] = b

def pbf_main():
    mapping(positions, old_positions)
    apply_gravity()
    
    neighbor_particles.fill(-1)
    find_neighbor_particles()
    for _ in range(pbf_iter):
        compute_lambdas()
        compute_delta_positions()
        apply_delta_positions()
        
    confine_to_boundary()
    update_velocities()

def rendering(gui):
    canvas = gui.canvas
    canvas.clear(bg_color)
    pos_np = positions.to_numpy()
    for pos in pos_np:
        for j in range(dim):
            pos[j] *= screen_to_world_ratio / screen_res[j]
    gui.circles(pos_np, radius=particle_radius, color=particle_color)
    gui.rect((0, 0), (board_states[None][0] / boundary[0], 1.0),
             radius=1.5,
             color=boundary_color)
    gui.show()

def init_particles():
    np_positions = np.zeros((num_particles, dim), dtype=np.float32)
    delta = kernel * 0.8
    num_x = num_particles_x
    num_y = num_particles // num_x
    assert num_x * num_y == num_particles
    offs = np.array([(boundary[0] - delta * num_x) * 0.5,
                     (boundary[1] * 0.02)],
                    dtype=np.float32)

    for i in range(num_particles):
        np_positions[i] = np.array([i % num_x, i // num_x]) * delta + offs
    np_velocities = (np.random.rand(num_particles, dim).astype(np.float32) -
                     0.5) * 4.0

    @ti.kernel
    def init(p: ti.ext_arr(), v: ti.ext_arr()):
        for i in range(num_particles):
            for c in ti.static(range(dim)):
                positions[i][c] = p[i, c]
                velocities[i][c] = v[i, c]

    @ti.kernel
    def init2():
        board_states[None] = ti.Vector([boundary[0] - epsilon, -0.0])

    init(np_positions, np_velocities)
    init2()

def main():
    init_particles()
    gui = ti.GUI('PBF2D', screen_res)
    while gui.running:
        move_board()
        pbf_main()
        rendering(gui)

if __name__ == '__main__':
    main()

[Taichi] Starting on arch=cuda
